# KOIOS
* Detecting the heart rate of a patient using signal processing and machine learning
* Project number: 87
* By Perry Tubul 205874290 and Eyal Golan 204229223

# Table of Contents
1. [Advanced engine implementation](#Advanced)
2. [Setup](#Setup)
3. [HR detection](#HR)
4. [Wavelet based Multivariate De-noising](#DWT)
5. [Conclusions](#conclusions)

# <a class="anchor" id="Advanced">1. Advanced engine implementation</a>
Using a similar logic as the basic engine, with the following changes for better results:
* Wavelet based Multivariate De-noising -
* MTCNN neural network - in order to detect the regions of intrest of a moving patient
* luminance detection and manipulation for more accurate heart rate results - used mainly when there are changing or bad
 lightning conditions

# <a class="anchor" id="Setup">2. Setup</a>
#### Initializing the Logger and the MTCNN neural networks, reading the videos location and detecting the OS

In [1]:
from facenet_pytorch import MTCNN #for detecting ROI of a moving subject
import torch
import face_recognition, PIL.Image, PIL.ImageDraw,math
import logging
import cv2
import platform
import scipy.signal as sig
import os
import pywt
from sklearn.decomposition import PCA
from sklearn.covariance import MinCovDet as MCD
import numpy as np
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import sys

FORMAT = '[%(asctime)s] [%(levelname)s] [%(funcName)s] [%(lineno)d] : %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

with open("input_path.txt") as input_file:
    input_location = input_file.read()
pattern = ".avi"
video_sources = []

for path, subdirs, files in os.walk(input_location):
    for name in files:
        if name.endswith(pattern):
            video_sources.append(os.path.join(path, name))

print(video_sources)
logging.info("Starting ...")
if platform.system() == "Windows":
    seperator = "\\"
else:
    seperator = "/"


# since MTCNN is a collection of neural nets and other code, the device must be passed
# in the following way to enable copying of objects when needed internally
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)


In [ ]:
#%run ./evm_preprocessing.ipynb
# video_location = dataset_location + specific_dir + seperator + "out.avi"
#video_location="out2.avi"

#### Detects face landmarks and parsing the ROI

In [ ]:
def detect_face(frame):
    """
    Detect face in a frame
    :param frame: A video frame
    :return: the location of the face in the picture
    """
    face_locations_handle_motion = mtcnn.detect(frame) # using pre-trained model to find faces
    face_location = list() 
    face_location.append(float(face_locations_handle_motion[0][0][1]))
    face_location.append(float(face_locations_handle_motion[0][0][2]))
    face_location.append(float(face_locations_handle_motion[0][0][3]))
    face_location.append(float(face_locations_handle_motion[0][0][0]))
    face_locations = list()
    face_locations.append(face_location)
    return face_locations

def parse_roi(frame):
    """
    Find a face and its region of interests.  
    @param frame: A video frame
    @return: None in case of which no face was detected. A tuple in a form of (forehead, nose_to_upper_lip) 
    forehead is ndarray that represents the subject's forehead, nose_to_upper_lip is ndarray that represents the region
    between the upper lip and the nose of the subject. 
    """
    # image = face_recognition.load_image_file(frame) # read image.
    face_locations = face_recognition.face_locations(frame,model = 'hog') # detects all the faces in image
    face_landmarks_list = face_recognition.face_landmarks(image)
    
    try:
        face_locations = detect_face(frame)
    except Exception as e:
        logging.error("Failure in face detection, error: " + str(e))
        
    # iterate through all the faces.
    for face_location in face_locations:
        img = PIL.Image.fromarray(frame)
        top,right,bottom,left = face_location # extract all face square points.
        diff = math.floor((top - bottom) * 0.15) # 20 percent of the face len (toadd eyebrow top point).
        
        # finding the forehead
        try:
            right_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['right_eyebrow']) # right eyebrow points.
        except Exception as e:
            logging.warning("No forehead found, " + str(e))
            return None
        right_eyebrow_landmarks.sort(axis=0)
        rightest_point = right_eyebrow_landmarks[-1] # The most right point of the ROI(according to x).
        top_right_eyebrow = right_eyebrow_landmarks.min(axis = 0)[1]
        try:
            left_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['left_eyebrow'])
        except Exception as e:
            logging.warning("No left eyebrow found, " + str(e))
            return None
        left_eyebrow_landmarks.sort(axis=0)
        leftest_point = left_eyebrow_landmarks[0] # the most left point of ROI.(according to x)
        top_left_eyebrow = left_eyebrow_landmarks.min(axis = 0)[1]
        bottom = min(top_right_eyebrow,top_left_eyebrow).item(0) # bottom point of the forehead.
        bottom = bottom - (0.05 * bottom) # improve bottom location by 2 percent.
        forehead = img.crop((leftest_point[0], leftest_point[1]+diff, rightest_point[0],bottom+10)) # adding diff to top to make the forehead bigger.

        # finding the second ROI:
        try:
            upper_mouth = np.asarray(face_landmarks_list[0]['top_lip']) # top_lip landmarks
        except Exception as e:
            logging.warning("No upper mouth found, " + str(e))
            return None
        upper_mouth_min = upper_mouth.min(axis = 0)[1] # The  top - lip upper point.
        try:
            upper_nose = np.asarray(face_landmarks_list[0]['nose_bridge'])
        except Exception as e:
            logging.warning("No upper nose found, " + str(e))
            return None
        upper_nose_min = upper_nose.min(axis = 0)[1]  # noise bridge upper point.
        upper_nose_min += upper_mouth_min * 0.1 # improving the noise bridge upper point.
        nose_to_upper_lip = img.crop((leftest_point[0], upper_nose_min, rightest_point[0], upper_mouth_min))

        return forehead, nose_to_upper_lip
    return None # in case of which no face was detected

## Check for bad frames
##### Criteria is:  R > 95 and G > 40 and B > 20 and R > G and R > B
##### Based on https://arxiv.org/ftp/arxiv/papers/1708/1708.02694.pdf page 5

In [ ]:
red_min_val = 95
green_min_val = 40
blue_min_val = 20
red_green_max_diff = 15
def good_frame(blue, green, red):
    """
        Checks the light conditions 
        @param: blue Is the mean of all the blue pixels in one frame
        @param: green Is the mean  of all the green pixels in one frame
        @param: red Is the mean of all the red pixels in one frame
        return False 0 meaning the lightning are bad, True when the lightning condition are OK
    """
    if red <= red_min_val:
        logging.warning("bad frame detected, reason: red > red_min_val")
        return False
    if green <= green_min_val:
        logging.warning("bad frame detected, reason: green > green_min_val")
        return False
    if blue <= blue_min_val:
        logging.warning("bad frame detected, reason: blue > blue_min_val")
        return False
    if red <= green:
        logging.warning("bad frame detected, reason: red > green")
        return False
    if red <= blue:
        logging.warning("bad frame detected, reason: red > blue")
        return False
    if abs(red - green) <= red_green_max_diff:
        logging.warning("bad frame detected, reason: abs(red - green) > red_green_max_diff")
        return False

    return True

In [ ]:
def get_new_frame(vidcap):
    """
    Reads new video frame and return it.
    @param vidcap: Pointer to the video 
    @return True, Image where there is more frames in the video to be read else False, None 
    """
    success, next_image = vidcap.read()
    return success, next_image

#### Plotting RGB arrays

In [ ]:
def plot_result(greens, reds, blues, x_value, title=""):
    """
    Generic function to plot graphs.
    @param greens: 1D array, greens[i] is the mean of all the green color in frame i 
    @param reds: 1D array, reds[i] is the mean of all the red color in frame i
    @param blues:1D array, blues[i] is the mean of all the blue color in frame i
    @param x_value: 1D array for the X- axis
    @param title: Title of the plot default is ""
    @return: None
    """
    logging.info("Plotting results ..." + title)
    plt.figure(figsize=(6, 5))
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.subplot(3, 1, 1)
    plt.plot(x_value, greens, color="green")
    plt.subplot(3, 1, 2)
    plt.plot(x_value, reds, color="red")
    plt.subplot(3, 1, 3)
    plt.plot(x_value, blues, color="blue")
    plt.show()
    logging.info("Showing result")

### Filtering

In [ ]:
def filter_channel(channel,fs):
    """
    This method apply filter on a channel between 0.75HZ to 4HZ.
    :param channel: Is a signal to apply the filter to.
    :param fs: Is the sampling rate of channel.
    :return: The filtered channel.
    """
    bh, ah = sig.butter(4, 0.75 / (fs / 2), 'highpass')
    bl, al = sig.butter(4, 4 / (fs / 2), 'lowpass')
    try:
        channel = sig.filtfilt(bh, ah, channel) # applying the filter coefficient on the sig
    except:
        return None
    #channel = np.absolute(channel)
    channel_after_filter = sig.filtfilt(bl, al, channel) # applying the filter coefficient on the sig
    return channel_after_filter

### Get RGB values from a frame
#### Check light frame light condition

In [ ]:
def parse_luminace(red, green, blue):
    """
    Calculate frame's luminace
    :param red: Mean value of all pixels in the red channel
    :param green: Mean value of all pixels in the green channel
    :param blue: Mean value of all pixels in the blue channel
    :return: The luminace value
    """
    luminance_level = 0.2126 * red + 0.7152 * green + 0.0722 * blue
    return luminance_level

def parse_RGB(rois, color_sig):
    """
    Calculate all the mean value of the green channel in ROI.
    :param roi: Area that we used to extract HR
    :param color_sig: is the the green signal thus far
    :return: False in case there is an error, color_sig and luminace value
    """
    for i,r in enumerate(rois):
        # extracting RGB colors from the frame
        red = r.getchannel(0)
        green = r.getchannel(1)
        blue = r.getchannel(2)
        b_mean,g_mean,r_mean = np.mean(blue),np.mean(green),np.mean(red)
        luminance_level = parse_luminace(r_mean, g_mean, b_mean)
        if good_frame(b_mean,g_mean,r_mean):
            #color_channels = r.reshape(-1, r.shape[-1])
            #avg_color = color_channels.mean(axis=0)
            color_sig[i].append(g_mean)
    return True, color_sig, luminance_level

In [ ]:
def log_video_details(video_source):
    """
    Logging current video information
    :param video_source: Location of the current video that is being processed.
    :return: None
    """
    logging.info("\nInformation on video:\t\t\t\t\t\t" + str(video_source) +
                 "\nFPS:\t\t\t\t\t\t" + str(fps) + 
                 "\nRound FPS:\t\t\t\t\t\t" + str(round_fps) + 
                 "\nNumber of frames:\t\t\t\t" + str(number_of_frames) + 
                 "\nNumber of bad frames:\t\t\t\t" + str(bad_frames) + 
                 "\nMax luminanace:\t\t\t\t\t" + str(max_luminance) + 
                 "\nMin luminanace:\t\t\t\t\t" + str(min_luminance) +
                 "\nMax diff of luminanace between adjacent frames:\t" + str(max_diff_luminance_adjacent) +
                 "\nAvg luminanace:\t\t\t\t\t" + str(avg_luminance))

In [ ]:
# <a class="anchor" id="DWT">4. Wavelet based Multivariate De-noising</a>
* The de-noising of the multivariate signal is performed by removing the noise elements ε by
thresholding the noisy signals
* The heart beat is shifted to a frequency with lower power in the power spectral distribution (PSD).
* We estimated the heart rate by exploiting the periodicity of the PPG signal by a frequency based voting scheme

In [ ]:
def indexes(y, thres=0.3, min_dist=1, thres_abs=False):
    """
    Finds the peaks in the signal
    :param y:  is the signal
    :param thres: minimum height of the peaks
    :param min_dist: minimum distance between each peek 
    :param thres_abs: 
    :return: array of all the peaks
    """
    if isinstance(y, np.ndarray) and np.issubdtype(y.dtype, np.unsignedinteger):
        raise ValueError("y must be signed")

    if not thres_abs:
        thres = thres * (np.max(y) - np.min(y)) + np.min(y)

    min_dist = int(min_dist)

    # compute first order difference
    dy = np.diff(y)

    # propagate left and right values successively to fill all plateau pixels (0-value)
    zeros, = np.where(dy == 0)

    # check if the signal is totally flat
    if len(zeros) == len(y) - 1:
        return np.array([])

    if len(zeros):
        # compute first order difference of zero indexes
        zeros_diff = np.diff(zeros)
        # check when zeros are not chained together
        zeros_diff_not_one, = np.add(np.where(zeros_diff != 1), 1)
        # make an array of the chained zero indexes
        zero_plateaus = np.split(zeros, zeros_diff_not_one)

        # fix if leftmost value in dy is zero
        if zero_plateaus[0][0] == 0:
            dy[zero_plateaus[0]] = dy[zero_plateaus[0][-1] + 1]
            zero_plateaus.pop(0)

        # fix if rightmost value of dy is zero
        if len(zero_plateaus) and zero_plateaus[-1][-1] == len(dy) - 1:
            dy[zero_plateaus[-1]] = dy[zero_plateaus[-1][0] - 1]
            zero_plateaus.pop(-1)

        # for each chain of zero indexes
        for plateau in zero_plateaus:
            median = np.median(plateau)
            # set leftmost values to leftmost non zero values
            dy[plateau[plateau < median]] = dy[plateau[0] - 1]
            # set rightmost and middle values to rightmost non zero values
            dy[plateau[plateau >= median]] = dy[plateau[-1] + 1]

    # find the peaks by using the first order difference
    peaks = np.where(
        (np.hstack([dy, 0.0]) < 0.0)
        & (np.hstack([0.0, dy]) > 0.0)
        & (np.greater(y, thres))
    )[0]

    # handle multiple peaks, respecting the minimum distance
    if peaks.size > 1 and min_dist > 1:
        highest = peaks[np.argsort(y[peaks])][::-1]
        rem = np.ones(y.size, dtype=bool)
        rem[peaks] = False

        for peak in highest:
            if not rem[peak]:
                sl = slice(max(0, peak - min_dist), peak + min_dist + 1)
                rem[sl] = True
                rem[peak] = False

        peaks = np.arange(y.size)[~rem]

    return peaks

def print_results(frames_window, window, xlabel, ylabel, change_range, title):
    """
    Generic function that plots signals for a 30 seconds window
    @param frames_window: 1D array represent the seconds in the window 
    @param window:HR 30 seconds signal 
    @param xlabel: label to be shown in the X - axis
    @param ylabel: label to be shown in the Y - axis
    @param change_range:
    @return: None
    """
    fig = plt.figure()
    fig.suptitle(title)
    ax = fig.subplots()
    while len(frames_window) > len(window):
        frames_window = frames_window[:-1]
    ax.plot(frames_window,window,color ='green')
    if change_range:
        plt.xlim([0, 5])
    # add some data to the signal 
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xscale('linear')
    ax.spines['bottom'].set_color('red')
    ax.spines['left'].set_color('red')
    ax.xaxis.label.set_color('red')
    ax.yaxis.label.set_color('red')
    ax.tick_params(axis='x', colors='red')
    ax.tick_params(axis='y', colors='red')
    plt.show()

def find_peaks_with_scipy(frames_window, window, round_fps, hr):
    """ 
    Find all the peaks in window
    :param frames_window: 1D array represent the seconds in the window 
    :param window: HR 30 seconds signal 
    :param round_fps: round frames per second
    :param hr: is the Heart rate 
    :return: None
    """ 
    while len(frames_window) > len(window):
        frames_window = frames_window[:-1]
    peaks, _ = sig.find_peaks(window, distance=round_fps)
    plt.plot(frames_window,window,'-go',markerfacecolor='red',markevery=peaks)
    plt.title("Peaks with scipy")
    plt.show()
    logging.info("Peaks vector with scipy: " + str(peaks) + " num of peaks: " + str(len(peaks)))

def print_peaks(frames_window, window):
    """
    Add a mark to all the peaks in window
    :param frames_window: 1D array represent the seconds in the window 
    :param window: 30 seconds signal 
    :return: 
    """
    while len(frames_window) > len(window):
        frames_window = frames_window[:-1]
    peaks = indexes(window,min_dist=20)
    plt.plot(frames_window,window,'-go',markerfacecolor='red',markevery=peaks)
    plt.title("Peaks with implemented function")
    plt.show()
    logging.info("Peaks vector by implemented function: " + str(frames_window[peaks]) + " num of peaks: " + str(len(frames_window[peaks])))
  
    
def find_hr_in_window(green, window_start, round_fps, window_id, window_size):
    """
    Split the signal into windows and find HR in each window
    :param green: is the signal of the all video
    :param window_start: index of where the window start in green
    :param round_fps: round frames per seconds
    :param window_id: window number
    :param window_size: size of the window - 30 seconds or less if there are not enough frames
    :return: None
    """
    round_fps = int(round_fps)
    if window_start + round_fps * window_size > len(green):
        window = green[window_start : ]
    else:
        window = green[window_start : window_start + round_fps * window_size]
    frames_window = np.arange(window.size/round_fps,step= (1/round_fps))
    #plot the window:
    print_results(frames_window, window, 'X-axis', 'Y-axis', False, "Green signal")
    # normalize window signal
    window = window - np.mean(window)
    window = window / np.std(window)
    print_results(frames_window, window, 'X-axis', 'Y-axis', False, "Green signal normalized")
    # filter the signal and plot results
    g = filter_channel(window,round_fps)
    if g is None: # An error occurred while filtering the signal
        return 
    print_peaks(frames_window, g)
    print_results(frames_window, window, 'X-axis', 'Y-axis', False, "")
    #plot the frequencies:
    f, Pxx_den = sig.periodogram(g, round_fps)
    print_results(f, Pxx_den, 'Frequency [Hz]', 'PSD [V**2/Hz]', True, "PSD by Frequency")
    # find the maximum freq.
    max_val = Pxx_den.argmax()
    logging.info("Window " + str(window_id) +
                 ":\nHighest freq:" + str(f[max_val]) + "\nHeart rate: " + str(f[max_val]*60))
    # find peaks - peak means breath
    find_peaks_with_scipy(frames_window, g,round_fps, f[max_val]*60)

In [11]:
np.set_printoptions(threshold=sys.maxsize)
def eigenvalue_decomposition(mat):
    """
    Apply eigenvalue decomposition of the matrix mat
    :param mat: the matrix to apply the decomposition to.
    :return: diagonal matrix which consist of the eigenvalues of mat eigen vectors
    """
    eigen_vals, eigen_vectors = np.linalg.eigh(mat)
    d = np.diag(eigen_vals)
    return d,eigen_vectors

    
def multivariate_video_signal(signal):
    """
    Implementation of the algorithm from the paper.
    :param signal: is a 2D signal. One dimension the signal from the forehead, and second is from above the upper lip ( 1 dimension for each region in rois)
    :return: multivariate PPG as described in the paper.
    """    
     # convert the signal into vector
    sig1 = signal[0,:].T
    sig2 = signal[1,:].T
    n = len(sig1)
    wavelet = pywt.Wavelet('sym2') 
    # apply wavlet decomposition:
    c = pywt.wavedec(np.column_stack((sig1,sig2)),wavelet,level = 4,axis = 0)
    mcd = MCD(random_state = 0).fit(c[-1]).covariance_ # noise matrix
    V,D,VT = np.linalg.svd(mcd) # noise matrix decomposition
    lst = []
    pca = PCA(1) 
    comp = StandardScaler().fit_transform(c[0]) # normalize the features
    c[0] = comp
    comp = pca.fit_transform(comp)
    comp = np.column_stack((comp,comp))
    lst.append(comp)
    # Basis change 
    for i in range (1,len(c)): # the denoising
        x_i = np.dot(c[i],V)
        gama_0 = np.sqrt(2*np.log(n)*D[0])
        xi_0 = pywt.threshold(x_i[:,0],value= gama_0,mode='soft')
        gama_1 = np.sqrt(2*np.log(n)*D[1])
        xi_1 = pywt.threshold(x_i[:,1],value= gama_1,mode='soft')
        xi = np.column_stack((xi_0,xi_1))
        xi = xi@VT
        lst.append(xi)
    return pywt.waverec(lst,wavelet,axis = 0) # reconstruct 2d array

def mvd(color_sig):
    """
    Perfor mmultivariatee de-noising
    :param color_sig: is the HR signal with noise 
    :return: De-noise HR rate signal
    """
    ppg_hat = multivariate_video_signal(color_sig)
    return ppg_hat


In [ ]:
def detect_hr(video_source):
    """
    Find HR estimation from the signal
    :param video_source: path to the video that is being procceesd
    :return: None
    """
    logging.info("\n=======================\n" + video_source + "\n=======================\n")
    color_sig_array = np.asarray(color_sig) # convert the signal to ndarray
    plt.figure(figsize=(15,6))
    plt.subplot(2,1,1)
    plt.plot(color_sig_array[0],"green")
    sig_mvd = mvd(color_sig_array) # apply the denoising
    # filtering the signals
    ppg_hat_zero = filter_channel(sig_mvd[:,0].T,round_fps) 
    ppg_hat_one = filter_channel(sig_mvd[:,1].T,round_fps)
    ppg_hat = np.vstack((ppg_hat_zero,ppg_hat_one))
    plt.subplot(2,1,2)
    green = ppg_hat[0] # choose the signal that was extracted from the subject's forehead.
    plt.plot(green,"red")
    # Split the signal into 30 seconds signals and find HR in each of them
    window_start = 0
    window_size = 30
    window_id = 0
    limit = good_frame_number - int(round_fps) * window_size
    while window_start < limit :
        find_hr_in_window(green, window_start, round_fps, window_id, window_size)
        window_start += int(round_fps) * window_size
        window_id += 1
    if window_start < good_frame_number:
        find_hr_in_window(green, window_start, round_fps, window_id, good_frame_number - window_start)

#### Main loop - going over all the frames of the video

In [ ]:
# Parsing video:
for video_location in video_sources:
    color_sig = [[],[]]
    heart_rates = []
    good_frame_number = 0
#     total_frame_number = 180
    total_frame_number = 0
    logging.info("Working on video " + video_location)
    vidcap = cv2.VideoCapture(video_location)
    success, image = vidcap.read()
    fps = vidcap.get(cv2.CAP_PROP_FPS) # fs == sampling rate
    round_fps = np.round(fps)
    number_of_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    logging.info("Parsing images ...")
    skipped_frames = 0
    bad_frames = 0
    max_luminance = 0
    min_luminance = 200
    avg_luminance = 0
    perv_luminance = None
    max_diff_luminance_adjacent = 0

    while success:
        if total_frame_number % 100 == 0:
            logging.info("Parsing frame " + str(total_frame_number) + "/" + str(number_of_frames))
        rois = parse_roi(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # build image ROI (rois is a tuple contains two regions)
        if rois is not None :
            try:
                is_good_frame,color_sig, luminance_level = parse_RGB(rois, color_sig) # add mean value of the  green channel in the current frame to the signal 
                if perv_luminance is not None and luminance_level - perv_luminance > max_diff_luminance_adjacent:
                    max_diff_luminance_adjacent = luminance_level - perv_luminance
                if luminance_level > max_luminance:
                    max_luminance = luminance_level
                if luminance_level < min_luminance:
                    min_luminance = luminance_level
                avg_luminance += luminance_level
                perv_luminance = luminance_level
            except Exception as e:
                logging.error("failed to get output from parse_RGB!\nError:" + str(e))
                is_good_frame = False
                bad_frames += 1
            if is_good_frame:
                good_frame_number += 1
                #logging.info("luminance level: " + str(luminance_level))
        if rois is None:
            bad_frames += 1
        total_frame_number += 1
        success, image = get_new_frame(vidcap)
    avg_luminance /= total_frame_number
    log_video_details(video_location)
    try:
        detect_hr(video_location) # Display the HR
    except Exception as e:
        logging.warning("Issue in detecting hr in video: " + str(e))